# TP3 - Encontrar el logotipo de la gaseosa

In [11]:
%matplotlib qt
import os
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

## Encontrar el logotipo de la gaseosa dentro de las imágenes provistas

1. Obtener una detección del logo en cada imagen sin falsos positivos

In [ ]:
# =============================================================================
# UTILITY FUNCTIONS
# =============================================================================

def load_image(path, max_size=1200):
    """Load image and return RGB, grayscale, and BGR versions."""
    img = cv.imread(path)
    h, w = img.shape[:2]
    if max(h, w) > max_size:
        scale = max_size / max(h, w)
        img = cv.resize(img, None, fx=scale, fy=scale)
    
    img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    return img_rgb, img_gray, img


def load_template(path, max_size=400):
    """Load template as grayscale."""
    template = cv.imread(path, 0)
    h, w = template.shape[:2]
    if max(h, w) > max_size:
        scale = max_size / max(h, w)
        template = cv.resize(template, None, fx=scale, fy=scale)
    return template


def create_template_variants(template):
    """
    Create multiple template variants to improve matching.
    Includes: grayscale, inverted, edges, binary, sobel, CLAHE, morphological, and Laplacian.
    """
    blurred = cv.GaussianBlur(template, (3, 3), 0)
    equalized = cv.equalizeHist(blurred)
    
    # CLAHE
    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_img = clahe.apply(blurred)
    
    # Binary (Otsu)
    _, binary = cv.threshold(equalized, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    
    # Edges
    edges = cv.Canny(equalized, 50, 150)
    
    # Sobel
    sobel_x = cv.Sobel(equalized, cv.CV_8U, 1, 0, ksize=3)
    sobel_y = cv.Sobel(equalized, cv.CV_8U, 0, 1, ksize=3)
    sobel = cv.addWeighted(sobel_x, 0.5, sobel_y, 0.5, 0)
    
    # Morphological
    kernel = np.ones((2, 2), np.uint8)
    dilated = cv.dilate(equalized, kernel, iterations=1)
    eroded = cv.erode(equalized, kernel, iterations=1)
    
    # Laplacian (from Clase4/04.Piramides)
    laplacian = cv.Laplacian(equalized, cv.CV_8U)
    
    # LOG (Laplacian of Gaussian)
    log = cv.Laplacian(blurred, cv.CV_8U)
    
    return {
        'original': equalized,
        'inverted': 255 - equalized,
        'edges': edges,
        'binary': binary,
        'binary_inv': 255 - binary,
        'sobel': sobel,
        'clahe': clahe_img,
        'clahe_inv': 255 - clahe_img,
        'dilated': dilated,
        'eroded': eroded,
        'laplacian': laplacian,
        'log': log,
    }


def draw_detections(img_rgb, detections):
    """Draw detection rectangles on image."""
    img_out = img_rgb.copy()
    for det in detections:
        x, y, w, h = det[:4]
        cv.rectangle(img_out, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return img_out


def plot_results_grid(results, title, save_path=None):
    """Plot results in a grid layout."""
    n = len(results)
    n_cols = 3
    n_rows = (n + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    axes = axes.flatten()
    
    for idx, (img_rgb, detections, img_name) in enumerate(results):
        img_result = draw_detections(img_rgb, detections)
        axes[idx].imshow(img_result)
        axes[idx].set_title(f"{img_name}\n{len(detections)} detection(s)")
        axes[idx].axis('off')
    
    for idx in range(n, len(axes)):
        axes[idx].axis('off')
    
    fig.suptitle(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=120, bbox_inches='tight')
    plt.show()


def plot_single_result(img_rgb, detections, title, save_path=None):
    """Plot single image result."""
    img_result = draw_detections(img_rgb, detections)
    plt.figure(figsize=(12, 10))
    plt.imshow(img_result)
    plt.title(f"{title} | {len(detections)} detections")
    plt.axis('off')
    if save_path:
        plt.savefig(save_path, dpi=120, bbox_inches='tight')
    plt.show()

In [ ]:
# =============================================================================
# DETECTION FUNCTIONS
# =============================================================================

def create_template_variants(template):
    """
    Create only useful template variants for template matching.
    Only: original, clahe, smooth, inverted.
    """
    smooth = cv.GaussianBlur(template, (3, 3), 0)
    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_img = clahe.apply(template)
    inverted = 255 - template
    
    return {
        'original': template,
        'clahe': clahe_img,
        'smooth': smooth,
        'inverted': inverted,
    }


def preprocess_image(img_gray):
    """Preprocess image before template matching."""
    img = cv.GaussianBlur(img_gray, (3, 3), 0)
    img = cv.equalizeHist(img)
    return img


def compute_iou(box1, box2):
    """Compute IoU between two boxes (x, y, w, h)."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)
    
    inter_w = max(0, xi2 - xi1)
    inter_h = max(0, yi2 - yi1)
    inter_area = inter_w * inter_h
    
    area1 = w1 * h1
    area2 = w2 * h2
    union_area = area1 + area2 - inter_area
    
    return inter_area / union_area if union_area > 0 else 0


def nms_global(detections, iou_threshold=0.3):
    """
    Apply Non-Maximum Suppression globally across all detections.
    detections: list of (x, y, w, h, score)
    """
    if not detections:
        return []
    
    detections = sorted(detections, key=lambda d: d[4], reverse=True)
    
    keep = []
    while detections:
        best = detections.pop(0)
        keep.append(best)
        detections = [d for d in detections if compute_iou(best[:4], d[:4]) < iou_threshold]
    
    return keep


def find_by_template(img_gray, template_variants, single_detection=True, min_threshold=0.30):
    """
    Multi-scale template matching.
    - 30 scales between 0.2 and 1.6
    - Adaptive threshold: mean + 2.5*std, min 0.30
    - Global NMS
    - Aspect ratio filtering: 0.5 < w/h < 3.0
    """
    h, w = img_gray.shape
    img_processed = preprocess_image(img_gray)
    
    scales = np.linspace(0.2, 1.6, 30)
    all_detections = []
    
    for scale in scales:
        for name, tmpl in template_variants.items():
            th, tw = tmpl.shape
            new_w = int(tw * scale)
            new_h = int(th * scale)
            
            if new_w > w or new_h > h or new_w < 20 or new_h < 10:
                continue
            
            scaled_tmpl = cv.resize(tmpl, (new_w, new_h))
            res = cv.matchTemplate(img_processed, scaled_tmpl, cv.TM_CCOEFF_NORMED)
            
            mean_val = np.mean(res)
            std_val = np.std(res)
            threshold = max(mean_val + 2.5 * std_val, min_threshold)
            
            locations = np.where(res >= threshold)
            
            for pt in zip(*locations[::-1]):
                x, y = pt
                score = res[y, x]
                
                aspect = new_w / new_h if new_h > 0 else 0
                if not (0.5 < aspect < 3.0):
                    continue
                
                all_detections.append((x, y, new_w, new_h, score))
    
    if not all_detections:
        return (None, None, 0) if single_detection else []
    
    final_detections = nms_global(all_detections, iou_threshold=0.3)
    
    if single_detection:
        best = final_detections[0]
        return (best[0], best[1], best[2], best[3]), 'template', best[4]
    
    return final_detections


def find_by_color_multi(img_bgr, min_area=500):
    """Find multiple Coca-Cola logos by color detection (for Assignment 2)."""
    h, w = img_bgr.shape[:2]
    hsv = cv.cvtColor(img_bgr, cv.COLOR_BGR2HSV)

    mask1 = cv.inRange(hsv, np.array([0, 70, 50]), np.array([10, 255, 255]))
    mask2 = cv.inRange(hsv, np.array([160, 70, 50]), np.array([180, 255, 255]))
    red_mask = cv.bitwise_or(mask1, mask2)
    red_mask = cv.morphologyEx(red_mask, cv.MORPH_CLOSE, np.ones((3, 3), np.uint8))

    white_mask = cv.inRange(hsv, np.array([0, 0, 180]), np.array([180, 50, 255]))
    white_on_red = cv.bitwise_and(white_mask, red_mask)
    dilated = cv.dilate(white_on_red, np.ones((5, 8), np.uint8), iterations=2)

    contours, _ = cv.findContours(dilated, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    detections = []
    for cnt in contours:
        x, y, cw, ch = cv.boundingRect(cnt)
        area = cw * ch
        aspect = cw / ch if ch > 0 else 0
        if aspect > 1.0 and area > min_area and cw > 30 and ch > 15:
            detections.append((x, y, cw, ch, area / (w * h)))
    return detections


def detect_logo(img_bgr, img_gray, template_variants, single_detection=True):
    """
    Detection pipeline.
    - Single detection: Template matching
    - Multi detection: Color-based detection
    """
    if not single_detection:
        return find_by_color_multi(img_bgr)

    bbox, method, score = find_by_template(img_gray, template_variants, single_detection=True)
    if bbox:
        return bbox, method, score

    return None, None, 0

In [ ]:
# =============================================================================
# TEMPLATE VARIANTS VISUALIZATION (4 useful transformations)
# =============================================================================

TEMPLATE_PATH = 'template/pattern.png'
template = load_template(TEMPLATE_PATH)
variants = create_template_variants(template)

fig, axes = plt.subplots(1, 4, figsize=(14, 4))

for ax, (name, img) in zip(axes, variants.items()):
    ax.imshow(img, cmap='gray')
    ax.set_title(name)
    ax.axis('off')

fig.suptitle(f'Template Variants ({len(variants)} transformations)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('results/template_variants.png', dpi=120, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# ASSIGNMENT 1: Single Detection per Image
# =============================================================================

IMAGES_DIR = 'images/'
image_files = sorted([f for f in os.listdir(IMAGES_DIR) if f.endswith(('.png', '.jpg'))])

results = []
for img_name in image_files:
    img_path = os.path.join(IMAGES_DIR, img_name)
    img_rgb, img_gray, img_bgr = load_image(img_path)
    
    bbox, method, match_val = detect_logo(img_bgr, img_gray, variants, single_detection=True)
    
    if bbox:
        x, y, w, h = bbox
        detections = [(x, y, w, h, 1.0)]
        print(f"{img_name}: {method} (match={match_val:.2f})")
    else:
        detections = []
        print(f"{img_name}: NOT DETECTED")
    
    results.append((img_rgb, detections, img_name))

plot_results_grid(results, "ASSIGNMENT 1: Single Detection per Image", 'results/Figure_1.png')

2. Plantear y validar un algoritmo para múltiples detecciones en la imagen coca_multi.png con el mismo témplate del ítem 1

In [ ]:
# =============================================================================
# ASSIGNMENT 2: Multiple Detections on coca_multi.png
# =============================================================================

img_rgb, img_gray, img_bgr = load_image('images/coca_multi.png')
detections = detect_logo(img_bgr, img_gray, variants, single_detection=False)

plot_single_result(img_rgb, detections, "ASSIGNMENT 2: coca_multi.png", 'results/Figure_2.png')

3. Generalizar el algoritmo del item 2 para todas las imágenes

In [ ]:
# =============================================================================
# ASSIGNMENT 3: Generalized Algorithm for All Images
# =============================================================================

results_single = []
results_multi = []

for img_name in image_files:
    img_rgb, img_gray, img_bgr = load_image(os.path.join(IMAGES_DIR, img_name))
    
    # Single detection
    bbox, method, _ = detect_logo(img_bgr, img_gray, variants, single_detection=True)
    det_single = [(bbox[0], bbox[1], bbox[2], bbox[3], 1.0)] if bbox else []
    results_single.append((img_rgb, det_single, img_name))
    
    # Multiple detection
    det_multi = detect_logo(img_bgr, img_gray, variants, single_detection=False)
    results_multi.append((img_rgb, det_multi, img_name))

plot_results_grid(results_single, "ASSIGNMENT 3: Single Detection", 'results/Figure_3a_single.png')
plot_results_grid(results_multi, "ASSIGNMENT 3: Multiple Detection", 'results/Figure_3b_multi.png')